In [ ]:
#urllib is used to downlaod the utils frile from deeplearning.net

import urllib.request
with urllib.request.urlopen("http://deeplearning.net/tutorial/code/utils.py") as url:
    response = url.read()
target = open('utils.py','w')
target.write(response.decode('utf-8'))
target.close()

In [1]:
#libraries
import math
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data
from utils import tile_raster_images

C:\Users\jssrv\_programs\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jssrv\_programs\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jssrv\_programs\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jssrv\_programs\Anaconda3\lib\site-packages\tensorflow\python\frame

In [2]:
#class that defines the behavious of the RBM
class RBM(object):
    def __init__(self,input_size, output_size):
        #defineing the hyperparameters
        self._input_size = input_size
        self._output_size = output_size
        self.epochs = 5 #epoch is a training iteration
        self.learning_rate = 1.0 #the step used in the gradiate descent
        self.batchsize = 100 #size of how much data will be used for training per sub iteration
        
        #initializing weights and biases as matrices full of zeroes
        self.w = np.zeros([input_size,output_size],np.float32)
        self.hb = np.zeros([output_size],np.float32) #creates ad ninitializes hidden bias
        self.vb = np.zeros([input_size],np.float32) #creates and initializes the visible bias
        
    #fits the results from the weighted visible layer plus the bias into a sigmoid curve
    def prob_h_given_v(self,visible,w,hb):
        #sigmoid
        return tf.nn.sigmoid(tf.matmul(visible,w)+hb)
    
    #fits the results from the weighted hidden layer plus the bias into a sigmoid curve
    def prob_v_given_h(self,hidden,w,vb):
        #sigmoid
        return tf.nn.sigmoid(tf.matmul(hidden, tf.transpose(w))+vb)
    
    #generates the sample probablility
    def sample_prob(self,probs):
        return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))
    
    #training method for the model
    def train(self,x):
        #create the placeholder for the parameters
        _w = tf.placeholder("float",[self._input_size,self._output_size])
        _hb = tf.placeholder("float",[self._output_size])
        _vb = tf.placeholder("float",[self._input_size])
        
        prv_w = np.zeros([self._input_size,self._output_size],np.float32) #initialize weights
        prv_hb = np.zeros([self._output_size],np.float32)
        prv_vb = np.zeros([self._input_size],np.float32)
        
        cur_w = np.zeros([self._input_size,self._output_size],np.float32)
        cur_hb = np.zeros([self._output_size],np.float32)
        cur_vb = np.zeros([self._input_size],np.float32)
        v0 = tf.placeholder("float32",[None,self._input_size])
        
        #initialize with smaple probabilities
        h0 = self.sample_prob(self.prob_h_given_v(v0,_w,_hb))
        v1 = self.sample_prob(self.prob_v_given_h(h0,_w,_vb))
        h1 = self.prob_h_given_v(v1,_w,_hb)
        
        #create the gradiates
        positive_grad = tf.matmul(tf.transpose(v0),h0)
        negative_grad = tf.matmul(tf.transpose(v1),h1)
        
        #update learning rates for the layers
        update_w = _w + self.learning_rate*(positive_grad - negative_grad)/tf.to_float(tf.shape(v0)[0])
        update_vb = _vb + self.learning_rate*tf.reduce_mean(v0 - v1,0)
        update_hb = _hb + self.learning_rate*tf.reduce_mean(h0 - h1,0)
        
        ##find error rate
        err = tf.reduce_mean(tf.square(v0-v1))
        
        #training loop
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            #for each epoch
            for epoch in range(self.epochs):
                #for each step/batch
                for start, end in zip(range(0,len(x),self.batchsize),range(self.batchsize,len(x),self.batchsize)):
                    batch = x[start:end]
                    cur_w = sess.run(update_w,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    cur_hb = sess.run(update_hb,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    cur_vb = sess.run(update_vb,feed_dict={v0:batch,_w:prv_w,_hb:prv_hb,_vb:prv_vb})
                    prv_w=cur_w
                    prv_hb=cur_hb
                    prv_vb=cur_vb
                error = sess.run(err,feed_dict={v0:x,_w:cur_w,_vb:cur_vb, _hb: cur_hb})
                print('Epoch: %d'%epoch,'reconstruction error: %f'% error)
            self.w = prv_w
            self.hb = prv_hb
            self.vb = prv_vb
            
    #create expected output for the DBN
    def rbm_output(self,x):
        input_x = tf.constant(x)
        _w = tf.constant(self.w)
        _hb = tf.constant(self.hb)
        out = tf.nn.sigmoid(tf.matmul(input_x,_w)+_hb)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            return sess.run(out)
            

In [4]:
#read in the data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

#asign the trainign and test data
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#create the deep belief network
rbm_hidden_sizes = [500,200,50]

inpx = trX
rbm_list = []
#size of the inputs is the number of inputs in the training set
input_size = inpx.shape[1]

for i,size in enumerate(rbm_hidden_sizes):
    print('RBM: ',i,'',input_size,'->',size)
    rbm_list.append(RBM(input_size,size))
    input_size = size


RBM:  0  784 -> 500
RBM:  1  500 -> 200
RBM:  2  200 -> 50


In [7]:
#train the BRM
for rbm in rbm_list:
    print('New RBM:')
    rbm.train(inpx)
    inpx = rbm.rbm_output(inpx)

New RBM:
Epoch: 0 reconstruction error: 0.060405
Epoch: 1 reconstruction error: 0.052860
Epoch: 2 reconstruction error: 0.048933
Epoch: 3 reconstruction error: 0.046987
Epoch: 4 reconstruction error: 0.046126
New RBM:
Epoch: 0 reconstruction error: 0.037183
Epoch: 1 reconstruction error: 0.032205
Epoch: 2 reconstruction error: 0.030102
Epoch: 3 reconstruction error: 0.028666
Epoch: 4 reconstruction error: 0.028206
New RBM:
Epoch: 0 reconstruction error: 0.052309
Epoch: 1 reconstruction error: 0.048687
Epoch: 2 reconstruction error: 0.048108
Epoch: 3 reconstruction error: 0.047539
Epoch: 4 reconstruction error: 0.047226


In [13]:
class NN(object):
    
    def __init__(self, sizes, X, Y):
        #Initialize hyperparameters
        self._sizes = sizes
        self._X = X
        self._Y = Y
        self.w_list = []
        self.b_list = []
        self._learning_rate =  1.0
        self._momentum = 0.0
        self._epoches = 10
        self._batchsize = 100
        input_size = X.shape[1]
        
        #initialization loop
        for size in self._sizes + [Y.shape[1]]:
            #Define upper limit for the uniform distribution range
            max_range = 4 * math.sqrt(6. / (input_size + size))
            
            #Initialize weights through a random uniform distribution
            self.w_list.append(
                np.random.uniform( -max_range, max_range, [input_size, size]).astype(np.float32))
            
            #Initialize bias as zeroes
            self.b_list.append(np.zeros([size], np.float32))
            input_size = size
      
    #load data from rbm
    def load_from_rbms(self, dbn_sizes,rbm_list):
        #Check if expected sizes are correct
        assert len(dbn_sizes) == len(self._sizes)
        
        for i in range(len(self._sizes)):
            #Check if for each RBN the expected sizes are correct
            assert dbn_sizes[i] == self._sizes[i]
        
        #If everything is correct, bring over the weights and biases
        for i in range(len(self._sizes)):
            self.w_list[i] = rbm_list[i].w
            self.b_list[i] = rbm_list[i].hb

    #Training method
    def train(self):
        #Create placeholders for input, weights, biases, output
        _a = [None] * (len(self._sizes) + 2)
        _w = [None] * (len(self._sizes) + 1)
        _b = [None] * (len(self._sizes) + 1)
        _a[0] = tf.placeholder("float", [None, self._X.shape[1]])
        y = tf.placeholder("float", [None, self._Y.shape[1]])
        
        #Define variables and activation functoin
        for i in range(len(self._sizes) + 1):
            _w[i] = tf.Variable(self.w_list[i])
            _b[i] = tf.Variable(self.b_list[i])
        for i in range(1, len(self._sizes) + 2):
            _a[i] = tf.nn.sigmoid(tf.matmul(_a[i - 1], _w[i - 1]) + _b[i - 1])
        
        #Define the cost function
        cost = tf.reduce_mean(tf.square(_a[-1] - y))
        
        #Define the training operation (Momentum Optimizer minimizing the Cost function)
        train_op = tf.train.MomentumOptimizer(self._learning_rate, self._momentum).minimize(cost)
        
        #Prediction operation
        predict_op = tf.argmax(_a[-1], 1)
        
        #Training Loop
        with tf.Session() as sess:
            #Initialize Variables
            sess.run(tf.global_variables_initializer())
            
            #For each epoch
            for i in range(self._epoches):
                
                #For each step
                for start, end in zip(range(0, len(self._X), self._batchsize), range(self._batchsize, len(self._X), self._batchsize)):
                    
                    #Run the training operation on the input data
                    sess.run(train_op, feed_dict={ _a[0]: self._X[start:end], y: self._Y[start:end]})
                
                for j in range(len(self._sizes) + 1):
                    #Retrieve weights and biases
                    self.w_list[j] = sess.run(_w[j])
                    self.b_list[j] = sess.run(_b[j])
                
                print ("Accuracy rating for epoch " + str(i) + ": " + str(np.mean(np.argmax(self._Y, axis=1) == sess.run(predict_op, feed_dict={_a[0]: self._X, y: self._Y}))))

In [16]:
nNet = NN(rbm_hidden_sizes,trX,trY)
nNet.load_from_rbms(rbm_hidden_sizes,rbm_list)
nNet.train()

Accuracy rating for epoch 0: 0.4767272727272727
Accuracy rating for epoch 1: 0.6291818181818182
Accuracy rating for epoch 2: 0.7335272727272727
Accuracy rating for epoch 3: 0.7728909090909091
Accuracy rating for epoch 4: 0.7914181818181818
Accuracy rating for epoch 5: 0.8032
Accuracy rating for epoch 6: 0.8154727272727272
Accuracy rating for epoch 7: 0.8438
Accuracy rating for epoch 8: 0.8756545454545455
Accuracy rating for epoch 9: 0.8982909090909091
